In [17]:
import pandas as pd
import json

args = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
args = pd.DataFrame(args)

In [18]:
corpus = sorted(set(args.titles.tolist()))
corpus

['. I am insensitive to the Gaming Industrys portrayal of women minorities and how it deals with other socially sensitive topics.',
 '. I believe in same sex and traditional marriage but not other types of marriage',
 '. I believe the CBS show Elementary Is worth more praise and acclaim than the BBC version Sherlock',
 'A Macbook is a better long term investment than a Windows computer.',
 'A Mexican Border Fence Would Be an Ineffective Deterrent and a Colossal Waste of Taxpayer Money and Resources',
 'A Wired Mouse is better than a Wireless mouse.',
 'A banana is not a good method of portraying scale.',
 'A comic book priced at . or . is disproportionately expensive to the amount of entertainment that can be garnered from it.',
 'A congressional reboot would breathe new life into the legislative branch of the federal U.S. government.',
 'A direct online democracy where people vote on issues instead of candidates would be better than always having to vote for the less bad of candidates

In [21]:
### COMMON SENSE TOPIC LABELS ###
import re

#TODOs: Prune and Add CauseNet Topics
cs_controversial_topics = [ln.strip() for ln in open("../../data/controversial_topics.txt")]
cs_debate_topics = [ln.strip() for ln in open("../../data/IBM_debator_topics.txt")]
cs_debate_topics_ = [ln.strip() for ln in open("../../data/IBM_debator_2.txt")]

topics = []
topics.extend(cs_controversial_topics)
topics.extend(cs_debate_topics)
topics.extend(cs_debate_topics_)

3900

In [32]:
# Semantic-Search, Cosine Similarity
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = topics
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = sorted(set(args.titles.tolist()))
arg_topics = dict((k, []) for k in queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(1, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        #print(corpus[idx], "(Score: {:.4f})".format(score))
        arg_topics[query].append(corpus[idx])

In [33]:
arg_topics

{'. I am insensitive to the Gaming Industrys portrayal of women minorities and how it deals with other socially sensitive topics.': ['Sexism in video gaming'],
 '. I believe in same sex and traditional marriage but not other types of marriage': ['Same-sex marriage'],
 '. I believe the CBS show Elementary Is worth more praise and acclaim than the BBC version Sherlock': ['Criticism of the BBC'],
 'A Macbook is a better long term investment than a Windows computer.': ['Criticism of Windows Vista'],
 'A Mexican Border Fence Would Be an Ineffective Deterrent and a Colossal Waste of Taxpayer Money and Resources': ['Mexico–United States barrier'],
 'A Wired Mouse is better than a Wireless mouse.': ['Wireless'],
 'A banana is not a good method of portraying scale.': ['Principle'],
 'A comic book priced at . or . is disproportionately expensive to the amount of entertainment that can be garnered from it.': ['Predatory pricing'],
 'A congressional reboot would breathe new life into the legislati

In [45]:
fout = open("../../data/claim_topics.jsonl", "w")

for k, v in arg_topics.items():
    fout.write(json.dumps({
        "claim": k,
        "topic": v
    }))
    fout.write("\n")